In [ ]:
import string
import keybert
from keybert import KeyBERT
from collections import Counter
import matplotlib.pyplot as plt
import bs4
import re
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)


In [ ]:
datafile = 'tweet_esp.json'

In [ ]:
import pandas as pd
tdf = pd.read_json(datafile)
tweets = tdf.transpose()
tweets.info()
tweets = tweets.assign(Time=pd.to_datetime(tweets.Datetime))

print("Number of tweets: ",len(tweets['Text']))
tweets.head(10)

In [ ]:
tweets['Text']

In [ ]:
# DATA CLEANING

tweet=[]
d=[]


CONTRACTIONS = {"aren't" : "are not", "can't" : "cannot", "couldn't" : "could not", "didn't" : "did not", "doesn't" : "does not",
                "don't" : "do not", "hadn't" : "had not", "hasn't" : "has not", "haven't" : "have not", "he'd" : "he had", "he'll" : "he will",
                "he's" : "he is", "i'd": "i had", "i'll" : "i will", "i'm" : "i am", "i've" : "i have", "isn't" : "is not","it'd" : "it had",
                "it'll" : "it will","it've" : "it have","it's" : "it is ","let's" : "let us", "mightn't" : "might not", "mustn't" : "must not",
                "shan't" : "shall not", "she'd" : "she had", "she'll" : "she will", "she's" : "she is", "shouldn't" : "should not", "that's" : "that is",
                "there's" : "there is", "they'd" : "they had","they'll" : "they will", "they're" : "they are", "they've" : "they have", "we'd" : "we had",
                "we're" : "we are", "we've" :  "we have", "weren't" : "were not", "what'll" : "what will", "what're" : "what are", "what's" : "what is", 
                "what've" : "what have", "where's" : "where is", "who'd" : "who had", "who'll" : "who will", "who're" : "who are", "who's" : "who is", 
                "who've" : "who have", "won't" : "will not", "wouldn't": "would not","you'd" : "you had","you'll" : "you will", "you're" : "you are",
                "you've" : "you have"}


'''
emoticons = {':-)' : "happy", ':)': "happy", ';)': "happy", ':o)': "happy", ':]': "happy", ':3': "happy", ':c)': "happy", ':>': "happy", '=]': "happy",
             '8)': "happy", '=)': "happy", ':}': "happy", ':^)': "happy", ':-D': "laughing", ':D': "laughing", '8-D': "laughing", '8D': "laughing", 
             'x-D': "laughing", 'xD': "laughing", 'X-D': "laughing", 'XD': "laughing", '=-D': "laughing", '=D': "laughing", 'xp': "laughing", 
             'XP': "laughing", ':-p': "laughing", ':p': "laughing", '=p': "laughing", ':-b': "laughing", ':L': "sad", ':-/': "sad", '>:/': "angry", 
             ':S': "sad", '>:[': "angry", ':@': "sad", ':-(': "sad", ':[': "sad", ':-||': "sad", '=L': "sad", ':<': "sad", ':-[': "sad", ':-<': "sad", 
             '=\\': "sad", '=/': "sad", '>:(': "angry", ':(': "sad", '>.<': "sad", ":'-(": "crying", ":'(": "crying", ':\\': "crying", ':-c': "sad", 
             ':c' : "angry", ':{' : "angry", '>:\\' : "angry", ';(' : "angry"}
'''




for i in range(len(tweets['Text'])):
   #Removing HTML tags      
        d = BeautifulSoup(tweets['Text'][i]).get_text() 


   #Replacing contractions
        d = d.replace("’","'") 
        words = d.split() 
        reformed = [CONTRACTIONS[word] if word in CONTRACTIONS 
                    else word for word in words]
        d = " ".join(reformed)
        

  #Removing hashtags
        d = ' '.join(re.sub("(@[A-Za-z0-9]+)|(#[A-Za-z0-9]+)", " ", d).split()) 


  #Removing URLs
        d = ' '.join(re.sub("(\w+:\/\/\S+)", " ", d).split()) 


  #Removing punctuations
        d = ' '.join(re.sub("[\.\,\!\?\:\;\-\=]", " ", d).split()) 


  #Removing Numbers
        pattern = r'[0-9]'
        d = ' '.join(re.sub(pattern, " ", d).split()) 


  #Removing characters
        d = ' '.join(re.sub("\W+", " ", d).split()) 


  #avoiding case sensitive issue
        d = d.lower() 

  
        tweet.append(d)
tweet[0:20]

In [ ]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [ ]:
kmodel = KeyBERT(model="paraphrase-multilingual-MiniLM-L12-v2")
keywords = kmodel.extract_keywords(tweet, keyphrase_ngram_range=(1, 1))

print(keywords)

In [ ]:
#storing keywords into a file
data_keywords = pd.DataFrame(keywords)
data_keywords.to_csv('keybert_keywords')

In [ ]:
#converting into list of tuples from list of list of tuples

single_list = [item for t in keywords for item in t]
print (single_list)


#only saving str values

final_list = [i[0] for i in single_list]
print(final_list)

In [ ]:

# Lemmatization
token = [lemmatizer.lemmatize(w) for w in final_list]

# Removing stopwords from the tokenized_text
processed_text = []

for word in token:
    if word not in stopwords.words('english'):
        processed_text.append(word)
            
print(processed_text)


In [ ]:
# Import library
text1 = ' '.join(cat for cat in processed_text)

import wordcloud
from wordcloud import WordCloud
# Instantiate Word Cloud
wc = WordCloud(width=2400,
               height=1500,
               min_font_size=10,
               background_color='white')
# Generate a word cloud
plt.figure(figsize = (24, 6))
keyword_wc = wc.generate(text1)
plt.xticks([])
plt.yticks([])
plt.imshow(keyword_wc)

In [ ]:
obj = SentimentIntensityAnalyzer()

In [ ]:
sentiment_dict = obj.polarity_scores(text1)
print(sentiment_dict)